In [4]:
# !pip install konlpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 18.5 MB/s eta 0:00:00


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load the uploaded train and dev datasets
train_data_path = '/content/drive/MyDrive/AI_Lab/AI_Lab_온라인강의/필수/NLP/대회/Code/data/train.csv'
dev_data_path = '/content/drive/MyDrive/AI_Lab/AI_Lab_온라인강의/필수/NLP/대회/Code/data/dev.csv'

train_df = pd.read_csv(train_data_path)
dev_df = pd.read_csv(dev_data_path)

# Display the first few rows of the train dataset for exploration (EDA)
print(train_df.head())
print(dev_df.head())

     fname                                           dialogue  \
0  train_0  #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...   
1  train_1  #Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...   
2  train_2  #Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...   
3  train_3  #Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...   
4  train_4  #Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...   

                                             summary     topic  
0  스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...   건강검진 받기  
1  파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...        백신  
2  #Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...     열쇠 찾기  
3  #Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...  여자친구가 있다  
4  말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...        댄스  
   fname                                           dialogue  \
0  dev_0  #Person1#: 안녕하세요, 오늘 하루 어떠셨어요? \n#Person2#: 요즘...   
1  dev_1  #Person1#: 헤이, 지미. 나중에 운동하러 가자.\n#Person2#: 그래...   
2  dev_2  #Person1#: 나는 더 이상 건

In [8]:
# Analyze dialogue length (word count) and number of participants

# Function to count number of participants in a dialogue
def count_participants(dialogue):
    return len(set([part.split('#')[1] for part in dialogue.split() if part.startswith('#Person')]))

# Add new columns for dialogue word count and number of participants
train_df['dialogue_word_count'] = train_df['dialogue'].apply(lambda x: len(x.split()))
train_df['num_participants'] = train_df['dialogue'].apply(count_participants)

# Basic statistics about the dialogue word count and number of participants
dialogue_stats = train_df[['dialogue_word_count', 'num_participants']].describe()

# Display the dataframe without ace_tools
print(dialogue_stats)

       dialogue_word_count  num_participants
count         12457.000000      12457.000000
mean             90.659469          2.012282
std              48.952439          0.133838
min              20.000000          2.000000
25%              58.000000          2.000000
50%              80.000000          2.000000
75%             113.000000          2.000000
max             606.000000          7.000000


EDA 결과
- 대화 길이: 평균 대화 길이는 약 91 단어이며, 대화의 최소 길이는 20 단어, 최대 길이는 606 단어입니다.
- 참여자 수: 대부분의 대화에서 2명이 대화를 나누고 있으며, 최소 2명에서 최대 7명이 대화에 참여합니다.

토큰화 작업

베이스 라인 코드에는 이미 Hugging Face의 AutoTokenizer를 사용하여 BART 모델에 맞게 토큰화 작업이 포함되어 있습니다. AutoTokenizer는 문장을 모델 입력에 맞게 적절한 토큰으로 변환하며, 패딩, 트렁케이션, 그리고 추가적인 special token을 처리해줍니다.